# Inference

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig

### Load Model

In [12]:
model_path = "./slang_interpret_model_v1"
print("Loading Model...")

config = PeftConfig.from_pretrained(model_path)
base_model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = PeftModel.from_pretrained(base_model, model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

print("Model Siap Digunakan!")

Loading Model...
Model Siap Digunakan!


### Inference & Test

In [13]:
def predict_slang(slang, context):
    input_text = f"jelaskan makna slang: {slang} | konteks: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_length=128,
            num_beams=4,
            repetition_penalty=1.2,
            early_stopping=True
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n=== UJI COBA MODEL ===")
test_cases = [
    ("mager", "duh ujan deres banget jadi mager parah mau kuliah"),
    ("baper", "dihh gtu aja lu udah baper"),
    ("bat", "wuih keren bat motor lu ngab"),
    ("ngab", "wuih keren bat motor lu ngab"),
    ("ngakak", "sumpah jokes lu receh banget bikin ngakak wkwk"),
    ("gercep", "info lomba dong kak, harus gercep nih"),
    ("utk", "ini krim bagus utk wajah"),
    ("njirr", "njirr lo mau nipu gue yak!"),
    ("gokil", "gila gokil banget performanya!")
]

for slang, ctx in test_cases:
    print(f"Slang   : {slang}")
    print(f"Konteks : {ctx}")
    print(f"Makna   : {predict_slang(slang, ctx)}")
    print("-" * 50)


=== UJI COBA MODEL ===
Slang   : mager
Konteks : duh ujan deres banget jadi mager parah mau kuliah
Makna   : 'mager' adalah bentuk gaul dari kata 'mager'. Menggunakan perubahan vokal untuk memberikan kesan lebih akrab atau 'imut'. Istilah ini digunakan untuk mengekspresikan emosi atau penekanan yang kuat.
--------------------------------------------------
Slang   : baper
Konteks : dihh gtu aja lu udah baper
Makna   : Istilah 'baper' memiliki arti 'bawa perasaan'. Merupakan bentuk singkatan untuk efisiensi percakapan teks.
--------------------------------------------------
Slang   : bat
Konteks : wuih keren bat motor lu ngab
Makna   : Dalam bahasa baku, 'bat' berarti 'banget'. Merupakan bentuk singkatan untuk efisiensi percakapan teks.
--------------------------------------------------
Slang   : ngab
Konteks : wuih keren bat motor lu ngab
Makna   : 'ngab' adalah bentuk gaul dari kata 'mengab'.
--------------------------------------------------
Slang   : ngakak
Konteks : sumpah jokes lu